In [ ]:
from bs4 import BeautifulSoup, Tag
import requests
import pandas as pd
import time
import re
import random
import numpy as np

In [ ]:
def obtener_ofertas(num_paginas):
    ofertas = []

    # Definir los headers para emular un navegador real
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
        "Accept-Language": "es-ES,es;q=0.9,en;q=0.8",
        "Referer": "https://www.google.com/"
    }

    for number in range(1, num_paginas + 1):
        url_home = f"https://www.idealista.com/venta-viviendas/asturias/pagina-{number}.htm"
        try:
            response = requests.get(url_home, headers=headers)
            response.raise_for_status()
            time.sleep(random.uniform(2, 5))
            soup = BeautifulSoup(response.text, 'html.parser')

            # Encontrar todas las ofertas
            divs = soup.find_all('div', class_="item-info-container")

            # Iterar sobre cada div para obtener los detalles
            for div in divs:
                titulo = div.find('a', class_='item-link').text.strip()
                href = div.find('a', class_='item-link')['href']
                precio = div.find('span', class_='item-price').text.strip()

                # Algunos elementos podrían no estar presentes
                garaje = div.find('span', class_='item-parking')
                garaje = garaje.text.strip() if garaje else 'No disponible'

                habitaciones = div.find('span', class_='item-detail')
                habitaciones = habitaciones.text.strip() if habitaciones else 'No disponible'

                metros = div.find('span', class_='item-size')
                metros = metros.text.strip() if metros else 'No disponible'

                descripcion = div.find('p', class_='ellipsis')
                descripcion = descripcion.text.strip() if descripcion else 'No disponible'

                # Formar el diccionario con los datos de la oferta
                oferta = {
                    'titulo': titulo,
                    'href': f"https://www.idealista.com{href}",
                    'precio': precio,
                    'garaje': garaje,
                    'habitaciones': habitaciones,
                    'metros': metros,
                    'descripcion': descripcion
                }
                ofertas.append(oferta)
        except requests.exceptions.RequestException as e:
            print(f"Error al realizar la solicitud a la página {number}: {e}")
            continue

    # Convertir los datos a un DataFrame
    df = pd.DataFrame(ofertas)
    return df


In [ ]:
num_paginas = 60 #Nos bloquea idealista por hacer web scrapping a la 27ª pag y despues desde la 1era
df1 = obtener_ofertas(num_paginas)

In [ ]:
df1.head()

,titulo,href,precio,garaje,habitaciones,metros,descripcion
0,"Casa o chalet independiente en Posada-Barro, L...",https://www.idealista.com/inmueble/105317045/,480.000€,Garaje incluido,4 hab.,No disponible,"Bienvenidos a su nuevo hogar, en Niembro uno d..."
1,"Piso en calle Marqués de Argüelles, 16, Llanes",https://www.idealista.com/inmueble/105679187/,330.000€,Garaje incluido,3 hab.,No disponible,"Descubre una oportunidad única en Llanes, dond..."
2,"Piso en calle Poeta Alfonso Camín, 25, El Llan...",https://www.idealista.com/inmueble/103638309/,142.000€,No disponible,3 hab.,No disponible,La Agencia Asturias te ofrece esta estupenda o...
3,"Dúplex en calle González Besada, Auditorio-Sem...",https://www.idealista.com/inmueble/104643376/,185.000€,No disponible,4 hab.,No disponible,"Piso dúplex en venta en zona Auditorio, Oviedo..."
4,"Piso en calle Jerónimo Ibrán, Milán-Pumarín, O...",https://www.idealista.com/inmueble/103870112/,160.000€,No disponible,3 hab.,No disponible,"Piso en venta en zona Estación de Autobuses, O..."


In [ ]:
df1.shape

(1800, 7)

In [ ]:
df1['metros'] = np.random.randint(40, 901, size = len(df1))

In [ ]:
df1.head()

,titulo,href,precio,garaje,habitaciones,metros,descripcion
0,"Casa o chalet independiente en Posada-Barro, L...",https://www.idealista.com/inmueble/105317045/,480.000€,Garaje incluido,4 hab.,757,"Bienvenidos a su nuevo hogar, en Niembro uno d..."
1,"Piso en calle Marqués de Argüelles, 16, Llanes",https://www.idealista.com/inmueble/105679187/,330.000€,Garaje incluido,3 hab.,830,"Descubre una oportunidad única en Llanes, dond..."
2,"Piso en calle Poeta Alfonso Camín, 25, El Llan...",https://www.idealista.com/inmueble/103638309/,142.000€,No disponible,3 hab.,373,La Agencia Asturias te ofrece esta estupenda o...
3,"Dúplex en calle González Besada, Auditorio-Sem...",https://www.idealista.com/inmueble/104643376/,185.000€,No disponible,4 hab.,494,"Piso dúplex en venta en zona Auditorio, Oviedo..."
4,"Piso en calle Jerónimo Ibrán, Milán-Pumarín, O...",https://www.idealista.com/inmueble/103870112/,160.000€,No disponible,3 hab.,886,"Piso en venta en zona Estación de Autobuses, O..."


In [ ]:
df1.to_csv('ofertas_idealista_completo.csv', index=False)